In [1]:
import os, sys
from datetime import datetime
import networkx as nx
from util import adjacency_matrix_to_nei_list, dirichlet_sampling_balanced, \
    get_adjacency_matrix, generate_node_configs, generate_attack_matrix
from torchvision.datasets import MNIST
from torchvision import transforms
from local_node import local_node
import pickle

Configs

In [16]:
num_peers = 10
alpha = 100
node_list = {}
maxRound = 5
maxEpoch = 3
dataset_name = "MNIST"
topology = "ring"

In [17]:
adj_matrix = get_adjacency_matrix(topology, num_peers)
nei_list = adjacency_matrix_to_nei_list(adj_matrix)
train_dataset = None
test_dataset = None

#define the attack
attack_type = 'label flipping'
targeted = False
poisoned_node_ratio = 20
noise_injected_ratio = 20
poisoned_sample_ratio = 10
attack_matrix = generate_attack_matrix(list(range(num_peers)), attack_type, targeted, poisoned_node_ratio, noise_injected_ratio, poisoned_sample_ratio)

# dataset
if dataset_name == "MNIST":
    train_dataset = MNIST(
        f"{sys.path[0]}/data", train=True, download=True, transform=transforms.ToTensor()
    )
    test_dataset = MNIST(
        f"{sys.path[0]}/data", train=False, download=True, transform=transforms.ToTensor()
    )

# datetime object containing current date and time
now = datetime.now()
dt_string = now.strftime("%d_%m_%Y_%H_%M_%S")
experimentsName = f'{num_peers}_clients_alpha_{alpha}_{dataset_name}_{topology}_'+dt_string
targets = train_dataset.targets
client_indices = dirichlet_sampling_balanced(targets, alpha, num_peers)
cwd = os.getcwd()

experimentsName_path = cwd+'/experiments/'+experimentsName
os.mkdir(experimentsName_path)


for client in range(num_peers):
    indices = client_indices[client]
    node_id = client      
    neiList = nei_list[client]
    
    attack_info = attack_matrix[node_id]
    attack_type = attack_info['attack_type']
    targeted = attack_info['targeted']
    noise_injected_ratio = attack_info['noise_injected_ratio']
    poisoned_sample_ratio = attack_info['poisoned_sample_ratio']

    node_config = generate_node_configs(node_id, indices, experimentsName, experimentsName_path, 
                                        dataset_name, neiList, num_peers, maxRound, maxEpoch, 
                                        train_dataset, test_dataset, attack_type, targeted, 
                                        noise_injected_ratio, poisoned_sample_ratio)
    
    basic_config = node_config['basic_config']
    data_train_loader = node_config['data_train_loader']
    data_val_loader = node_config['data_val_loader']
    test_dataset_loader = node_config['test_dataset_loader']
    
    node = local_node(node_id,basic_config, data_train_loader, data_val_loader, test_dataset_loader)
    node_list[node_id] = node
    
    with open(experimentsName_path+f"/{node_id}_config.pk", "wb") as f:
        pickle.dump(node_config, f)
        f.close()
    

for round in range(maxRound):
    for node_id in node_list:
        node = node_list[node_id]
        node.curren_round = round+1
        node.local_training()
        for nei in node.neiList:
            node_list[nei].add_nei_model(round+1, node_id, node.model)
    
    for node_id in node_list:
        node = node_list[node_id]
        node.aggregation()
        

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 4070 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Missing logger folder: d:\git\beidou/experiments/10_clients_alpha_100_MNIST_ring_03_06_2024_16_51_13\node_0
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name          | Type                      | Params | In sizes | Out sizes
-----------------------------------------------------------------------------------
0 | train_metrics | MetricCollection          | 0      | ?        | ?        
1 | val_metrics   | MetricCollection          | 0      | ?        | ?        
2 | test_m

Epoch end: Validation, epoch number: 0
Epoch end: Validation, epoch number: 1
Epoch end: Train, epoch number: 0
Epoch end: Validation, epoch number: 2
Epoch end: Train, epoch number: 1


`Trainer.fit` stopped: `max_epochs=3` reached.
You are using a CUDA device ('NVIDIA GeForce RTX 4070 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Epoch end: Validation, epoch number: 3
Epoch end: Train, epoch number: 2
Performance of Node 0 before aggregation at round 1


d:\git\beidou\venv\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:430: PossibleUserWarning: The dataloader, test_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


...

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 4070 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Node 9 aggregate model with [0, 8]
Performance of Node 9 after aggregation at round 5


Testing: 0it [00:00, ?it/s]

Epoch end: Test, epoch number: 8


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃   Runningstage.testing    ┃                           ┃
┃          metric           ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       Test/Accuracy       │    0.9402123093605042     │
│       Test/F1Score        │     0.933936357498169     │
│         Test/Loss         │    0.21147607266902924    │
│      Test/Precision       │    0.9428250193595886     │
│        Test/Recall        │    0.9402123093605042     │
│    TestEpoch/Accuracy     │    0.9370903372764587     │
│     TestEpoch/F1Score     │    0.9370183348655701     │
│    TestEpoch/Precision    │    0.9395456910133362     │
│     TestEpoch/Recall      │    0.9370903372764587     │
└───────────────────────────┴───────────────────────────┘